# テキスト処理の機械学習におけるBi-Normal Separation(BNS)の効果について   
## 背景  
日本語テキストを機械学習で分類する機会があった。データは簡単に以下のような感じ。
<p>
<!-- <table border="1"> -->
<table border="3">
<tr>
<td>入力データ</td> <td align="center">ある観点</td>
</tr>
<tr>
<td>日本語テキスト1</td> <td align="center">O</td> 
</tr>
<tr>
<td>日本語テキスト2</td> <td align="center">X</td>
</tr>
<tr>
<td>日本語テキスト3</td> <td align="center">O</td> 
</tr>
</table>
</p>  
- 入力データの入力とある観点のO/X付けは別の人が行っている
- "ある観点"は実際には複数あり、マルチラベルになっているがプロトタイプだったのでシングルラベルで処理   
- データ数は少ない(３桁後半) 
- データサイズは日本語文字数で500から800
- ポジティブデータの割合は小さい。観点によるがひと桁%〜30%  

当初、言語処理100本ノックで作ったプログラム(Cabocha,Python,scikit-learn。第５章、第６章、第８章)を修正して処理を行ったが精度が上がらなかった。そこでフィーチャー(素性)の選択について調べている中でBi-Normal Separation(BNS)を知り適用してみたところ顕著な改善が見られた。  
そのままはアップロードできないので言語処理100本ノックの第８章をベースにBNSの効果を見てみることにした。

## Bi-Normal Separation(BNS)について  
BNSについては以下の[レポート][1]を参考にした。  
> BNS Feature Scaling: An Improved Representation over TF·IDF for SVM
> Text Classification
> George Forman
> HP Laboratories
> HPL-2007-32R1  
 
上記レポートの勝手な要約。
- BNSでフィーチャー･スケーリングする場合フィーチャー数を減らす必要は無い  
- 減らす場合はInformation Gainで選択し、BNSでスケーリングするのが良い  

1点目が良い。更に、以下の特長も良い。

- 実装は楽(上記レポートのp2の右上でBNS関連の７行を実装できれば良い)

## 前提  
言語処理100本ノックをベースとするのでLogisticRegressionを使用してみる。  

また、以下の処理結果を比較。  
1. フィーチャーを抽出して、あるフィーチャーがあるレコードに登場する回数でスケーリング。上記の通りLogisticRegressionで処理(基本的な手法)  
1. 1.と同じフィーチャーを使い、BNSでスケーリング。 1.と同じLogisticRegressionで処理(BNSで処理)  

## 結果
結果は上から accuracy_score()、confusion_matrix(),classification_report()の出力。accuracyベースで7%の向上。日本語で処理した際程差がつかなかった。。。思いつく違いは以下の通り。  
- データ自体  
- データ数  
- ポジ/ネガの割合。前回はポジティブデータの割合は少ない。今回はほぼ５割(上記レポートでも、ポジティブデータは通常少ない、という記載がある)。  
- トレーニング用データとテスト用データの割合(今回は8:2。前回は9:1)

BNSケースも良くはないが今回はBNSの効果検証のためここまで。

なお、データ数を少なくして処理をしてみたがその際の結果は、基本的な手法の精度が上がり、BNSはこれを上回ったが＋２%程度であった。

### 基本的な手法での結果
```
0.7525
 [[810 267]
 [261 795]]
             precision    recall  f1-score   support

   positive       0.76      0.75      0.75      1077
   negative       0.75      0.75      0.75      1056

avg / total       0.75      0.75      0.75      2133
```
### BNSでの処理結果
```
0.8223
 [[885 192]
 [187 869]]
             precision    recall  f1-score   support

   positive       0.83      0.82      0.82      1077
   negative       0.82      0.82      0.82      1056

avg / total       0.82      0.82      0.82      2133
```
### TF-IDFでの処理結果
基本的な手法とBNSのケースは同じシャッフル結果を使用しているが、TF-IDFは異なるシャッフル結果を使用している。
```
0.7843
 [[851 226]
 [234 822]]
             precision    recall  f1-score   support

   positive       0.78      0.79      0.79      1077
   negative       0.78      0.78      0.78      1056

avg / total       0.78      0.78      0.78      2133
```
## その他  
お世話になったサイト。
- [言語処理100本ノック 2015][2]
- 言語処理100本ノックで使われているデータ。Bo Pang氏とLillian Lee氏が公開しているMovie Review Dataのsentence polarity dataset v1.0が使われている。サイトは[こちら][3]
- [scikitの様々なページ][4] 
- [githubのscikitのページ][5]。ストップワードはここから。  
 
[1]: http://www.hpl.hp.com/techreports/2007/HPL-2007-32R1.pdf?ref=driverlayer.com/web
[2]: http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch8 
[3]: http://www.cs.cornell.edu/people/pabo/movie-review-data/ 
[4]: http://scikit-learn.org/stable/index.html
[5]: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/stop_words.py

# 事前処理
## 文字コード確認と変換
久しぶりなので文字コードの確認から。その後iconvを使ってUTF-8に変換する。head/tailでおざなりだが確認。

In [ ]:
#!chardetect ./rt-polarity/*

In [ ]:
#!iconv -c -s --verbose -f WINDOWS-1252 -t UTF-8 ./rt-polarity/rt-polarity.pos > ./rt-polarity/rt-polarity.pos.utf8
#!iconv -c -s --verbose -f WINDOWS-1252 -t UTF-8 ./rt-polarity/rt-polarity.neg > ./rt-polarity/rt-polarity.neg.utf8

In [ ]:
#!chardetect ./rt-polarity/*.utf8

In [ ]:
#!head ./rt-polarity/rt-polarity.pos.utf8

In [ ]:
#!head ./rt-polarity/rt-polarity.neg.utf8

In [ ]:
#!tail ./rt-polarity/rt-polarity.pos.utf8

In [ ]:
#!tail ./rt-polarity/rt-polarity.neg.utf8

## テキストデータの事前処理
1. テキストデータのロード 
1. y(正解データ)準備
1. checkWord()でクリーニング。
1. stopwordsを処理
1. stemming処理

lower()は念の為。

In [ ]:
import numpy as np
import re

### checkWord()の処理 
- アルファベットだけ。そのまま返す
- 記号1文字。抜く。
- 記号(アルファベット以外のキャラクター)始まりでアルファベット。記号を抜く
- アルファベット始まりで記号終わり。記号を抜く
- 's。'sを抜く。
- 'd。'dを抜く。
- 'm。'mを抜く。。
- 've。'veを抜く。。
- 're。'reを抜く。。
- 'll。'llを抜く。。
- n't。そのまま。doesn'tとかそのまま。
- director/co-writerもそのまま。/は抜きたいが-は抜きたくない。しかしルールなさそう。
- その他気になりつつも抜かないもの。 
d'etre fun-for-fun's-sake cam'ron children's-movie it's-surreal bull's-eye so-bad-it's-funny life-at-arm's-length early-'80s o'neill mid-'70s c'mon matrix'-style stalk'n'slash sub'-standard soul's-eye so-bad-it's-good o'fallon so-bad-they're-good pasach'ke show-don't-tell o'clock can't-miss shoot-'em-up

まだ抜いた方が良さそうなものもあるが数は少なそうなので放置。

In [ ]:
def checkWord(_l):
#     from IPython.core.debugger import Pdb; Pdb().set_trace()
    retword=""
    alphaonly = r'^[a-zA-Z]+$'
    singlesymbol = r'^[^a-zA-Z]$'
    alphasecond = r'^([^a-zA-Z]+)'
    alphafirst = r'[^a-zA-Z]+$'
    lasts = r'\'s$'
    lastd = r'\'d$'
    lastm = r'\'m$'
    lastve = r'\'ve$'
    lastre = r'\'re$'
    lastll = r'\'ll$'
    
    for _w in _l:
        if re.match(alphaonly, _w):
            retword = _w
        elif re.match(singlesymbol,_w):
            retword = ""
        else:
            retword = re.sub(alphasecond,"",_w)
            retword = re.sub(alphafirst,"",retword)
            retword = re.sub(lasts,"",retword)
            retword = re.sub(lastd,"",retword)
            retword = re.sub(lastm,"",retword)
            retword = re.sub(lastve,"",retword)
            retword = re.sub(lastre,"",retword)
            retword = re.sub(lastll,"",retword)
        yield retword

### データのロードと正解データの作成

In [ ]:
posdata = np.loadtxt("./rt-polarity/rt-polarity.pos.utf8",delimiter="\n",dtype=str)
negdata = np.loadtxt("./rt-polarity/rt-polarity.neg.utf8",delimiter="\n",dtype=str)
y_pos = np.ones((posdata.shape[0]))
y_neg = np.zeros((negdata.shape[0]))


# posdata = np.loadtxt("./rt-polarity/rt-polarity.pos.utf8",delimiter="\n",dtype=str)
# negdata = np.loadtxt("./rt-polarity/rt-polarity.neg.utf8",delimiter="\n",dtype=str)
# # posdata = posdata[0:140]
# # negdata = negdata[0:700]
# posdata = shuffle(posdata, n_samples=140, random_state=0)
# negdata = shuffle(negdata, n_samples=700, random_state=0)
# y_pos = np.ones((posdata.shape[0]))
# y_neg = np.zeros((negdata.shape[0]))
# print("{} {}".format(posdata.shape,negdata.shape))

# import math
# posdata = np.loadtxt("./rt-polarity/rt-polarity.pos.utf8",delimiter="\n",dtype=str)
# negdata = np.loadtxt("./rt-polarity/rt-polarity.neg.utf8",delimiter="\n",dtype=str)
# y_neg = np.zeros((negdata.shape[0]))
# numP = math.ceil( y_neg.shape[0]*0.2 )
# posdata = posdata[0:numP]
# y_pos = np.ones((posdata.shape[0]))
# posdata.shape

### checkWordの呼び出し=クリーニング
Xとなるデータはここではリストに入れておく。

In [ ]:
filtereddata = [[],[]]

for _i,_d in  enumerate([posdata,negdata]):
    for _l in _d:
        _l=_l.lower()
        _l=_l.split()
        _ol = ""
        _ol = [w for w in checkWord(_l) if w != ""]
        _ol = " ".join(_ol)
        filtereddata[_i].append(_ol)

### ストップワード処理

In [ ]:
stopwords=[
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]
# from IPython.core.debugger import Pdb; Pdb().set_trace()
afterStopWords = [[],[]]
for _n,_d in  enumerate(filtereddata):
    for _l in _d:
        afterStopWords[_n].append( [w for w in _l.split() if not w in stopwords] )


### ステミング処理

In [ ]:
from nltk import stem

stemmedWords=[[],[]]
stemmer = stem.PorterStemmer()

# from IPython.core.debugger import Pdb; Pdb().set_trace()
for _n,_d in  enumerate(afterStopWords):
    for _l in _d:
        stemmedWords[_n].append( [stemmer.stem(w) for w in _l] )

### reストップワード処理
再びストップワード処理。
これまでの流れだと、例えば"gives"はストップワード処理に該当しない。その上でステミングで"give"になる。"give"はストップワードにリストされているのでもう一度処理しておく。

In [ ]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
re_afterStopWords = [[],[]]
for _n,_d in  enumerate(stemmedWords):
    for _l in _d:
        re_afterStopWords[_n].append( [w for w in _l if not w in stopwords] )


In [ ]:
stemmedWords[0]

### データのシャッフルと分割

In [ ]:
data = re_afterStopWords[0] + re_afterStopWords[1]
y_pos_neg = np.r_[y_pos,y_neg]

In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(data,y_pos_neg, random_state=20180729)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_txt, X_test_txt, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("{} {} {} {}".format(len(X_train_txt),len(X_test_txt),y_train.shape,y_test.shape))

# 基本的な手法 

## フィーチャー抽出
X_train_txtを対象にフィーチャー抽出

In [ ]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
words=[]
for _n,_l in enumerate(X_train_txt):
    words = words + [w for w in _l]

## 重複排除
- フィーチャー作成 
- フィーチャーを減らすことは考えない。 

In [ ]:
from collections import Counter 
c = Counter(words)
c.most_common()
len(c.most_common())
features=list(c)
len(features)
# features
print("{} {} {}".format(len(features),len(X_train_txt),len(X_test_txt)))

## データ作成
### データ保持用アレイ作成
### トレーニング用データとテスト用データ作成
出現回数をフィーチャーの値として使用。

In [ ]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
X_train = np.zeros((len(X_train_txt), len(features)))
X_test = np.zeros((len(X_test_txt), len(features)))
_X = [X_train,X_test]
for _i,_d in  enumerate([X_train_txt,X_test_txt]):
    for _n,_l in enumerate(_d):
        for _w in _l:
            try:
                _X[_i][_n][features.index(_w)] += 1
            except ValueError:
                pass
#                 print("_i:{} _n:{} _l:{} _w:{}".format(_i,_n,_l,_w))

In [ ]:
X_train

## sklearnでLogisticRegression
### sklearn関連関数

In [ ]:
def sklearnLR(_X_train,_y_train):
    from sklearn import linear_model
    # logreg = linear_model.LogisticRegressionCV()
    _model = linear_model.LogisticRegression()
    _model.fit(_X_train, _y_train)
    return _model

In [ ]:
def sklearnScorePredict(_model,_X_test,_y_test):
    _score=_model.score(_X_test,_y_test)
    _pred=_model.predict(_X_test)
    return _pred

In [ ]:
def sklearnCheckResult(_y_test,_pred):
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    print("{:.4f}\n {}".format(
        accuracy_score(_y_test, _pred),
        confusion_matrix(_y_test, _pred)
    ))
    target_names = ['positive', 'negative']
    print(classification_report(_y_test, _pred, target_names=target_names))

### LogisticRegression呼び出し

In [ ]:
logreg = sklearnLR(X_train,y_train)

In [ ]:
pred = sklearnScorePredict(logreg,X_test,y_test) 

In [ ]:
sklearnCheckResult(y_test,pred)

# BNSで処理

## メモ
$
\begin{align}
|F^{-1}(tpr)-F^{-1}(fpr)| \\
tpr = tp/pos \\
fpr = fp/neg
\end{align}
$

pos ポジティブのトレーニングケース数  
neg ネガティブのトレーニングケース数  
tp あるワードを含むポジティブトレーニングケース  
fp あるワードを含むネガティブトレーニングケース  

stemmedWords[0]にポジティブのトレーニングデータ。  
stemmedWords[1]にネガティブのトレーニングデータ。

## トレーニングデータ数確認

In [ ]:
pos = sum(y_train)
neg = y_train.shape[0] - pos
print("{} = {} + {} ".format(y_train.shape[0],pos,neg))

## TPとFP算出

In [ ]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
tplist=[]
fplist=[]
worklist=[]
# worklist = [1 for _r in re_afterStopWords[0] if "kafka" in _r]
for _n,_w in enumerate(features):
    worklist = [1 for _r in re_afterStopWords[0] if _w in _r]
#     worklist = [1 for _r in X_train_txt if _w in _r]
    tplist.append(sum(worklist))
    worklist = [1 for _r in re_afterStopWords[1] if _w in _r]
#     worklist = [1 for _r in X_test_txt if _w in _r]
    fplist.append(sum(worklist))
    if _n % 400 == 0:
        print("{} {} {} {}".format(_n,_w,tplist[_n],fplist[_n]))


## TPRとFPR算出
TPFとFPRを算出  
- tpf(or fpr)の値が1だと後続でエラー(ppt()で-inf)となるのでtpr値=pos数の場合tprを-0.00001しておく(ある単語が全レコードにあるとこのケースに該当する。極めて稀なケースだが念の為)。
- tpf(or fpr)の値が0だと後続でエラー(同上)となるので0より大きい値を代入。tprlistでは0より大きい値は0.00023391812865497077なので0.00001を代入する。
- tmpが並ぶのが恥ずかしいが。。。

In [ ]:
tmp1 = [x if x!=pos else x-0.00001 for x in tplist]
tmp2 = [x if x!=neg else x-0.000011 for x in fplist]
tmp3 = list(map(lambda x: x/pos,tmp1))
tmp4 = list(map(lambda x: x/neg,tmp2))
tprlist  = [max(0.00001,x) for x in tmp3]
fprlist  = [max(0.00001,x) for x in tmp4]

## BNS値算出

In [ ]:
from scipy.stats import norm
tmp7 = [norm.ppf(x)-norm.ppf(y) for x,y in zip(tprlist,fprlist)]
bns = list(map(lambda x: abs(x),tmp7))


## BNSスケーリングのもとでLogisticRegression

In [ ]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()
B_X_train = np.zeros((len(X_train_txt), len(features)))
B_X_test = np.zeros((len(X_test_txt), len(features)))
_X = [B_X_train,B_X_test]
for _i,_d in  enumerate([X_train_txt,X_test_txt]):
    for _n,_l in enumerate(_d):
        for _w in _l:
            try:
                _X[_i][_n][features.index(_w)] = bns[features.index(_w)]
            except ValueError:
#                 print("_i:{} _n:{} _l:{} _w:{}".format(_i,_n,_l,_w))
#                 print("_i:{} _n:{} _w:{}".format(_i,_n,_w))
                pass

In [ ]:
B_logreg = sklearnLR(B_X_train,y_train)
B_pred = sklearnScorePredict(B_logreg,B_X_test,y_test) 
sklearnCheckResult(y_test,B_pred)

# TF-IDFで処理
scikitの[Working With Text Data][0]に記載されたコード。

[0]: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

tfi_data = np.r_[posdata,negdata]
tfi_y_pos_neg = np.r_[y_pos,y_neg]

tfi_X, tfi_y = shuffle(tfi_data,tfi_y_pos_neg, random_state=20180729)
tfi_X_train_txt, tfi_X_test_txt, tfi_y_train, tfi_y_test = train_test_split(tfi_X, tfi_y, test_size=0.2, random_state=42)
tfi_X_train_txt[0]

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tfi_X_train_txt)
# X_train_counts.shape
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

clf = MultinomialNB().fit(X_train_tfidf, tfi_y_train)

X_new_counts = count_vect.transform(tfi_X_test_txt)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

tfi_predicted = clf.predict(X_new_tfidf)

sklearnCheckResult(y_test,tfi_predicted)